# Lesson 3, Part A: Show the data as it is acquired

In this lesson, we'll show how to use the tools provided with Bluesky to show the data as it is acquired using both a table representation and a graphical view, as well. These capabilities are provided by using callbacks. In lessons 1 and 2, we wrote our own simple callback to view the documents that come from the RunEngine during execution of a plan. Quickly, the data became too complex for simple viewing.

The *LiveTable* and *LivePlot* callbacks provide a table and graphical view of the data from the plan. We'll get to those first. Later, we'll show the *BestEffortCallback*, which combines both those callbacks plus a little more. For routine work, we'll want to use *BestEffortCallback* all the time. We'll show how to make that happen so we *set it and forget about it*.

-------

**note**:  This tutorial expects to find an EPICS IOC on the local network configured as a synApps [`xxx`](https://github.com/epics-modules/xxx) IOC with prefix `sky:`.  A docker container is available to provide this IOC.  See this URL for instructions:  https://github.com/prjemian/epics-docker/blob/master/n3_synApps/README.md

Starting with the configuration from lessons 1 and 2, we first group the `import`s together as is common Python practice:

In [1]:
from ophyd import EpicsMotor
from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp
from apstools.devices import use_EPICS_scaler_channels

ModuleNotFoundError: No module named 'ophyd'

Next, make a RunEngine (for scanning) and connect our motor and scaler

In [2]:
RE = RunEngine({})

P = "sky:"
m1 = EpicsMotor(f"{P}m1", name="m1")
scaler = ScalerCH(f"{P}scaler1", name="scaler")

NameError: name 'RunEngine' is not defined

Reconfigure the scaler for channel names, set the counting time to 0.5 s, and read the scaler values.

In [3]:
scaler.channels.chan01.chname.put("clock")
scaler.channels.chan02.chname.put("I0")
scaler.channels.chan03.chname.put("scint")

scaler.preset_time.put(0.4)

scaler.select_channels(None)
scaler.read()

NameError: name 'scaler' is not defined

## Showing the data

In lessons 1 and 2, we wrote a *callback* routine that printed information as the scan prpogressed (that is, we *printed select content from the stream of documents emitted by the RunEngine while executing a plan*).  But our callback was simple and we found there is a lot of content in the documents from the RunEngine.

The [simplest example of a Bluesky callback](http://nsls-ii.github.io/bluesky/callbacks.html#simplest-working-example) is the `print` function.  We want a callback function that understands our data and uses reasonable assumptions to show that data as it is being acquired.

One method to display our data is in a [table](http://nsls-ii.github.io/bluesky/callbacks.html#livetable) that updates as the scan progresses.  We'll import the `LiveTable` callback from the Bluesky library:

In [4]:
from bluesky.callbacks import LiveTable

ModuleNotFoundError: No module named 'bluesky'

`LiveTable()` shows acquired data as a plan is executed.  The argument is the list of detectors to show in the table.  First, we'll count the scaler 5 times.

In [5]:
RE(bp.count([scaler], 5), LiveTable([scaler]))

NameError: name 'RE' is not defined

You see columns for the data collection sequence number, the time of collection, and each of the *named* scaler channels.  At the end, the short form for the scan's `uid` is shown as well as `scan num` which is a more convenient reference to the scan.  The user has control to set or reset `scan num` so do not rely on that number to be unique.

Next, we'll scan with motor and scaler, as we did in lesson 2, displaying the acquired data in a `LiveTable`.

In [6]:
RE(bp.scan([scaler], m1, 1, 5, 5), LiveTable([m1, scaler]))

NameError: name 'RE' is not defined

In addition to the data columns from `count` above, the motor position (both where the motor reported as its position and where the motor was told to go, respectively) are shown.

---------
There is a callback routine that will plot the data as it is acquired.  When starting graphics, it is necessary to first initialize the graphics manager of the display.  The setup is specific to the graphics manager.  For command line or python program use, see http://nsls-ii.github.io/bluesky/callbacks.html#aside-making-plots-update-live.

For jupyter notebooks:

In [7]:
%matplotlib notebook
from bluesky.utils import install_nb_kicker
install_nb_kicker()

ModuleNotFoundError: No module named 'matplotlib'

We'll import the `LivePlot` callback from the Bluesky library:

In [8]:
from bluesky.callbacks import LivePlot

ModuleNotFoundError: No module named 'bluesky'

Count the scaler 5 times.  We'll just plot the `scint` signal.

In [9]:
RE(bp.count([scaler],num=5), LivePlot("scint"))

NameError: name 'RE' is not defined

To scan, we need to tell `LivePlot` to plot `scint' *vs.* the motor:

In [10]:
RE(bp.scan([scaler], m1, 1, 5, 5), LivePlot("scint", "m1"))

NameError: name 'RE' is not defined

------------------
Both the table and the plot are very useful diagnostics for routine use.  They have been combined in the [*Best-Efforts Callback*](http://nsls-ii.github.io/bluesky/callbacks.html#best-effort-callback) which provides best-effort plots and visualization for any plan.  It uses [user-configurable information](http://nsls-ii.github.io/bluesky/callbacks.html#hints) that is part of every ophyd device to make reasonable assumptions about what information is appropriate to display in the context of the current plan.

We'll import the `BestEffortCallback` callback from the Bluesky library:

In [11]:
from bluesky.callbacks.best_effort import BestEffortCallback

ModuleNotFoundError: No module named 'bluesky'

Count the scaler 5 times:

In [12]:
RE(bp.count([scaler], num=5), BestEffortCallback())

NameError: name 'RE' is not defined

You see both the `LiveTable` and the `LivePlot` output tangled up here in the jupyter notebook.  Each is created on demand and then updated as the plan progresses.  When executing in a command line environment, the `LivePlot` is shown in a separate window.

Repeat the same scan, noting that we do not need to inform the callback what to display:

In [13]:
RE(bp.scan([scaler], m1, 1, 5, 5), BestEffortCallback())

NameError: name 'RE' is not defined

Because this is such a useful tool, we want to make this callback happen all the time.  The RunEngine manages a list of such callbacks.  We *subscribe* the `BestEffortCallback`:

In [14]:
RE.subscribe(BestEffortCallback())

NameError: name 'RE' is not defined

Repeat the count of the scaler (without adding the callback in the command):

In jupyter notebook, we can see the `LiveTable` after our scan command.  To see the `LivePlot`, we have to look up a few cells, where the plots of scaler channels *vs.* `m1` are shown, our latest data identified by `scan num` in the legend.RE(bp.count([scaler], num=5))

In [15]:
RE(bp.count([scaler], num=5))

NameError: name 'RE' is not defined

In jupyter notebook, we can see the `LiveTable` after our count command.  To see the `LivePlot`, we have to look up a few cells, where the plots of scaler channels *vs.* *time* are shown, our latest data identified by `scan num` in the legend.

Then, repeat the scan (again, without adding the callback in the command):

In [16]:
RE(bp.scan([scaler], m1, 1, 5, 5))

NameError: name 'RE' is not defined

In jupyter notebook, we can see the `LiveTable` after our scan command.  Notice that the number of columns displayed is less than when we called `LiveTable` ourselves.  To see the `LivePlot`, we have to look up a few cells, where the plots of scaler channels *vs.* `m1` are shown, our latest data identified by `scan num` in the legend.

## Summary

We'll show this code as a python program:

```
#!/usr/bin/env python

"lesson 3: Show the data as it is acquired"

from ophyd import EpicsMotor
from ophyd.scaler import ScalerCH
from bluesky import RunEngine
import bluesky.plans as bp
from bluesky.callbacks import LiveTable
from bluesky.callbacks import LivePlot
from bluesky.callbacks.best_effort import BestEffortCallback
from apstools.devices import use_EPICS_scaler_channels


%matplotlib notebook
from bluesky.utils import install_qt_kicker
install_qt_kicker()


RE = RunEngine({})

P = "sky:"
m1 = EpicsMotor(f"{P}m1", name="m1")
scaler = ScalerCH(f"{P}scaler1", name="scaler")
m1.wait_for_connection()
scaler.wait_for_connection()
scaler.preset_time.put(0.4)
scaler.select_channels(None)
print(scaler.read())

RE(bp.count([scaler], num=5), LiveTable([scaler]))
RE(bp.scan([scaler], m1, 1, 5, 5), LiveTable([m1, scaler]))

RE(bp.count([scaler], num=5), LivePlot("scint"))
RE(bp.scan([scaler], m1, 1, 5, 5), LivePlot("scint", "m1"))

RE(bp.count([scaler], num=5), BestEffortCallback())
RE(bp.scan([scaler], m1, 1, 5, 5), BestEffortCallback())

RE.subscribe(BestEffortCallback())

RE(bp.count([scaler], num=5))
RE(bp.scan([scaler], m1, 1, 5, 5))
```